In [2]:
################################
## 데이터 전처리

import pandas as pd
import numpy as np
import random,datetime
from openpyxl import load_workbook

# 엑셀 파일 경로
file_path = ''

# 엑셀 파일 읽기
# wb = load_workbook(file_path)
time_slots = pd.read_excel(r'C:\Users\user\PoiTransport\근무시간정보.xlsx')
drivers = pd.read_excel(r'C:\Users\user\PoiTransport\운전자정보.xlsx')
buses = pd.read_excel(r'C:\Users\user\PoiTransport\버스정보.xlsx')

# 결측값 대체

time_slots.bfill()
drivers.bfill()
buses.bfill()

# 각 시트 데이터 확인
print(time_slots.head())
print(drivers.head())
print(buses.head())



          날짜  시간  순번  월   일  주차 요일
0 2024-08-20  오전   1  8  20  34  화
1 2024-08-20  오전   2  8  20  34  화
2 2024-08-20  오전   3  8  20  34  화
3 2024-08-20  오전   4  8  20  34  화
4 2024-08-20  오전   5  8  20  34  화
   운전자ID 운전자이름      연락처 근무조  직급
0      1   양귀선  010-111   A  주임
1      2   안병준  010-111   A  사원
2      3   김현진  010-111   A  사원
3      4   이명재  010-111   A  사원
4      5   이동규  010-111   A  사원
   버스ID  버스번호
0     1  9371
1     2  9370
2     3  9317
3     4  3354
4     5  3288


In [4]:
################################
## 배차 알고리즘

def Naive_assign_shifts(time_slots, drivers, buses): ## 단순한 배차 알고리즘. 교대조 고려 X
    driver_ids = drivers['운전자이름'].tolist()
    bus_ids = buses['버스번호'].tolist()
    
    # 공평하게 배차를 위해 무작위로 섞음
    np.random.shuffle(driver_ids)
    np.random.shuffle(bus_ids)
    
    assignments = []
    
    for index, row in time_slots.iterrows():
        driver_id = driver_ids[index % len(driver_ids)]
        bus_id = bus_ids[index % len(bus_ids)]
        assignments.append({
            '날짜': row['날짜'],
            '시간': row['시간'],
            '순번': row['순번'],
            '운전자ID': driver_id,
            '버스ID': bus_id
        })
    
    return pd.DataFrame(assignments)

def driver_filtering(row,last_shift, eligible_drivers): ## 2교대 근무자 필터링
    # 전날 오후 근무한 운전자는 다음 날 오전 근무 제외
    if row['시간'] == '오전':
        new_eligible_drivers = []
        for driver_id in eligible_drivers:
            day_delta =  datetime.date(row['날짜'].year,row['월'],row['일'])- datetime.date(last_shift[driver_id]['날짜'].year,last_shift[driver_id]['날짜'].month,last_shift[driver_id]['날짜'].day)
            if not ((last_shift[driver_id]['시간'] == '오후') and (day_delta.days == 1)):
                new_eligible_drivers.append(driver_id)
        eligible_drivers = new_eligible_drivers.copy()
    
    # 이미 같은 날 근무가 있는 운전자는 근무에서 제외
    eligible_drivers = [
        driver_id for driver_id in eligible_drivers
        if not (last_shift[driver_id]['날짜'] ==  row['날짜'])
    ]
    return eligible_drivers

def V1_assign_shifts(time_slots, drivers, buses):   ## 2교대 근무 자동배차알고리즘
    driver_groups = drivers.groupby('근무조')
    driver_ids = drivers['운전자ID']
    bus_ids = buses['버스ID'].tolist()

    work_count = {driver_id: 0 for driver_id in driver_ids}
    last_shift = {driver_id: {'시간': '오전', '날짜': datetime.date(1999,1,1)} for driver_id in driver_ids}
    
    # 운전자를 두 그룹으로 나눔. group_c 는 전체 운전자
    group_a , group_b ,group_c = driver_groups.get_group('A')['운전자ID'] , driver_groups.get_group('B')['운전자ID'], driver_ids
    assignments = []
    
    # 날짜,시간별 근무갯수 테이블(2024.8.20 기준 평일 오전6대 오후6대 체재)
    max_workcount_table = time_slots.groupby(['날짜','시간'])['순번'].max()

    switch_trig = False
    for index, row in time_slots.iterrows():

        # 주말,국경일 배차를 자동배차에서 제외
        if row['날짜'].weekday() in [5,6,7]: ## 5:토요일 6:일요일 7:국경일(별도지정필요)
            assignments.append({
                '날짜': row['날짜'],
                '시간': row['시간'],
                '월':row['월'],
                '일':row['일'],
                '주차':row['주차'],
                '요일':row['요일'],
                '순번': row['순번'],
                '운전자ID': 'N/A',
                '버스ID': 'N/A'
            })
            continue
        
        # 교대주기 트리거
        if row['날짜'].weekday() == 4:
            switch_trig = True
        # 주말 이후로 교대 근무 변경
        if(switch_trig):
            if row['날짜'].weekday() == 0:  # 월요일이 시작되는 경우
                group_a , group_b = group_b , group_a
                switch_trig = False

        if row['시간'] == '오전':
            eligible_drivers = group_a.copy()
        else:
            eligible_drivers = group_b.copy()
        
        ## 근무자 필터링
        eligible_drivers = driver_filtering(row,last_shift,eligible_drivers)

        # 근무자가 부족한 경우 다른 교대조도 포함하여 근무자 재필터링
        if (len(eligible_drivers) < 1):
            print(len(eligible_drivers),max_workcount_table[row['날짜'],row['시간']],' !!!기사 부족!!!')
            eligible_drivers = driver_filtering(row,last_shift,group_c)


        # 근무가능한 운전자가 없는 경우와 있는 경우
        if not len(eligible_drivers):
            assignments.append({
                '날짜': row['날짜'],
                '시간': row['시간'],
                '월':row['월'],
                '일':row['일'],
                '주차':row['주차'],
                '요일':row['요일'],
                '순번': row['순번'],
                '운전자ID': 'N/A',
                '버스ID': 'N/A'
            })
            continue
        else:
            # 근무 횟수가 적은 운전자 선택
            min_work_count = min([work_count[driver_id] for driver_id in eligible_drivers])
            least_busy_drivers = [
                driver_id for driver_id in eligible_drivers if work_count[driver_id] == min_work_count
            ]
            
            if least_busy_drivers:
                driver_id = np.random.choice(least_busy_drivers)
                bus_id = np.random.choice(bus_ids)
                
                assignments.append({
                    '날짜': row['날짜'],
                    '시간': row['시간'],
                    '월':row['월'],
                    '일':row['일'],
                    '주차':row['주차'],
                    '요일':row['요일'],
                    '순번': row['순번'],
                    '운전자ID': driver_id,
                    '버스ID': bus_id
                })
                
                work_count[driver_id] += 1
                last_shift[driver_id] = {'시간': row['시간'], '날짜': row['날짜']}
            else:
                assignments.append({
                '날짜': row['날짜'],
                '시간': row['시간'],
                '월':row['월'],
                '일':row['일'],
                '주차':row['주차'],
                '요일':row['요일'],
                '순번': row['순번'],
                '운전자ID': 'N/A',
                '버스ID': 'N/A'
            })
    print(work_count)
    return pd.DataFrame(assignments)



In [5]:
################################
# 배차 수행
schedule = V1_assign_shifts(time_slots, drivers, buses)



{1: 36, 2: 36, 3: 36, 4: 36, 5: 36, 6: 36, 7: 31, 8: 31, 9: 30, 10: 31, 11: 31, 12: 31, 13: 31}


In [6]:
################################
# 배차결과를 엑셀 파일로 저장
output_path = r'C:\Users\user\PoiTransport\배차시간표.xlsx'
schedule.to_excel(output_path, index=False)

print("배차 시간표가 성공적으로 생성되었습니다.")



배차 시간표가 성공적으로 생성되었습니다.


In [5]:
################################
## 배차시간표를 시각화된 엑셀로 변환.

from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment

# 파일 경로 지정
schedule_path = r'C:\Users\user\poi\PoiTransport\배차시간표.xlsx'
drivers_path = r'C:\Users\user\poi\PoiTransport\운전자정보.xlsx'

# 엑셀 파일 읽기
schedule = pd.read_excel(schedule_path)
drivers = pd.read_excel(drivers_path)

# 운전자ID를 운전자 이름으로 대체
schedule = schedule.merge(drivers[['운전자ID', '운전자이름']], on='운전자ID', how='left')
schedule.drop('운전자ID', axis=1, inplace=True)
schedule = schedule.rename(columns={'운전자이름': '운전자'})

# 운전자 이름 결측값 ''로 대체
schedule.fillna(value='',inplace=True)

def create_calendar_excel_format1(schedule):
    # 운전자 이름을 날짜와 시간에 맞춰서 피벗
    pivot_table = schedule.pivot(index= '순번', columns=['날짜','월','일','요일','시간'], values='운전자')
    
    # 운전자 이름이 인덱스인 테이블
    # pivot_table = schedule.pivot_table(index='운전자', columns=['날짜', '시간'], aggfunc=len, fill_value='')
    # 날짜 순서대로 열 정렬
    pivot_table = pivot_table.sort_index(axis=1, level=[0, 1])

    # 새로운 엑셀 파일로 저장
    return pivot_table

# 달력 형식으로 변환
calendar_format1 = create_calendar_excel_format1(schedule)

sorted_calendar_format = calendar_format1.apply(lambda x: sorted(x))

sorted_df = calendar_format1.apply(lambda x: sorted(x))

# 파일 경로 지정
output_path = r'C:\Users\user\poi\PoiTransport\달력배차표.xlsx'

# 새로운 워크북 생성
wb = Workbook()
ws = wb.active
ws.title = "배차 시간표"

# 달력 데이터 추가
calendar_columns = sorted_calendar_format.columns
L = len(calendar_columns)
for i in range(L//14):  ## 7일 매일 오전 오후 = 7*2 
    tmp_df = sorted_calendar_format[calendar_columns[i*14:(i+1)*14]]
    for r in dataframe_to_rows(tmp_df, index=True, header=True):
        ws.append(r)
    ws.append([])
tmp_df = sorted_calendar_format[calendar_columns[(L//14)*14:]]
for r in dataframe_to_rows(tmp_df, index=True, header=True):
    ws.append(r)

# 스타일 정의
center_aligned_text = Alignment(horizontal='center', vertical='center')
bold_font = Font(bold=True)
red_font = Font(color="FF0000", bold=True)

# 스타일 적용
max_index = sorted_calendar_format.index.__len__()  ## 최대 순번

for i, column in enumerate(sorted_calendar_format.columns, start=0):
    # 셀 지정
    date = column[0]
    date_cell = ws.cell(row= 1 + (max_index + 7)*(i//14), column = 2 + (2*(i//2))%14)  
    month_cell = ws.cell(row= 2 + (max_index + 7)*(i//14), column = 2 + (2*(i//2))%14)   ## 7:(날짜,월,일,요일,시간,순번빈칸,주간빈칸), 14: 오전오후*7 
    day_cell = ws.cell(row= 3 + (max_index + 7)*(i//14), column= 2 + (2*(i//2))%14)
    weekday_cell = ws.cell(row= 4 + (max_index + 7)*(i//14), column = 2 + (2*(i//2))%14)

    if date.weekday() >= 5:  # 5: 토요일, 6: 일요일
        date_cell.font = red_font
        month_cell.font = red_font
        day_cell.font = red_font
        weekday_cell.font = red_font

    else:
        date_cell.font = bold_font
        month_cell.font = bold_font
        day_cell.font = bold_font
        weekday_cell.font = bold_font

    #셀 스타일 적용
    for j in range(1,1 + max_index + 7):  ## 7:(날짜,월,일,요일,시간,순번빈칸)
        cell = ws.cell(row= j + (max_index + 7)*(i//14), column = 2 + i%14) 
        cell.alignment = center_aligned_text
    
    #순번셀 정렬
    if(i%14 == 0):
        for j in range(1,1 + max_index + 7):  ## 7:(날짜,월,일,요일,시간,순번빈칸)
            cell = ws.cell(row= j + (max_index + 7)*(i//14), column = 1) 
            cell.alignment = center_aligned_text


# 열 너비 조정
for col in range(1, 8):
    ws.column_dimensions[get_column_letter(col)].width = 12

# 엑셀 파일로 저장
wb.save(output_path)

print("달력 형식의 배차 시간표가 성공적으로 생성되었습니다.")




달력 형식의 배차 시간표가 성공적으로 생성되었습니다.


C:\Users\user\AppData\Local\Temp\ipykernel_12552\4271008141.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  schedule.fillna(value='',inplace=True)


In [25]:
################################
## 배차시간표를 시각화된 엑셀로 변환.

from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment

# 파일 경로 지정
schedule_path = r'C:\Users\user\poi\PoiTransport\배차시간표.xlsx'
drivers_path = r'C:\Users\user\poi\PoiTransport\운전자정보.xlsx'

# 엑셀 파일 읽기
schedule = pd.read_excel(schedule_path)
drivers = pd.read_excel(drivers_path)

# 운전자ID를 운전자 이름으로 대체
schedule = schedule.merge(drivers[['운전자ID', '운전자이름']], on='운전자ID', how='left')
schedule.drop('운전자ID', axis=1, inplace=True)
schedule = schedule.rename(columns={'운전자이름': '운전자'})

schedule.fillna(value='',inplace=True)

def create_calendar_excel_format2(schedule):
    # 운전자 이름을 날짜와 시간에 맞춰서 피벗
    ns = schedule.drop(['날짜','월','주차','버스ID'],axis = 1)
    pivot_table = ns.pivot(index= '순번', columns=['일','요일','시간'], values='운전자')
    
    # 운전자 이름이 인덱스인 테이블
    # pivot_table = schedule.pivot_table(index='운전자', columns=['날짜', '시간'], aggfunc=len, fill_value='')
    # 날짜 순서대로 열 정렬
    pivot_table = pivot_table.sort_index(axis=1, level=[0, 1])

    # 새로운 엑셀 파일로 저장
    return pivot_table

# 달력 형식으로 변환
calendar_format2 = create_calendar_excel_format2(schedule)

sorted_calendar_format = calendar_format2.apply(lambda x: sorted(x))

print(sorted_calendar_format)

# 파일 경로 지정
output_path = r'C:\Users\user\poi\PoiTransport\달력배차표2.xlsx'

# 새로운 워크북 생성
wb = Workbook()
ws = wb.active
ws.title = "배차 시간표"

# 달력 데이터 추가
calendar_columns = sorted_calendar_format.columns
L = len(calendar_columns)
print(L)
for i in range(L//28):  ## 14일 매일 오전 오후 = 14*2 
    tmp_df = sorted_calendar_format[calendar_columns[i*28:(i+1)*28]]
    for r in dataframe_to_rows(tmp_df, index=True, header=True):
        ws.append(r)
    ws.append([])
tmp_df = sorted_calendar_format[calendar_columns[(L//28)*28:]]
for r in dataframe_to_rows(tmp_df, index=True, header=True):
    ws.append(r)

# 스타일 정의
center_aligned_text = Alignment(horizontal='center', vertical='center')
bold_font = Font(bold=True)
red_font = Font(color="FF0000", bold=True)

# 스타일 적용
max_index = sorted_calendar_format.index.__len__()  ## 최대 순번

for i, column in enumerate(sorted_calendar_format.columns, start=0):
    # 셀 지정
    date = column[0]
    day_cell = ws.cell(row= 3 + (max_index + 5)*(i//28), column= 2 + (2*(i//2))%28)
    weekday_cell = ws.cell(row= 4 + (max_index + 5)*(i//28), column = 2 + (2*(i//2))%28)

    if date.weekday() >= 5:  # 5: 토요일, 6: 일요일
        day_cell.font = red_font
        weekday_cell.font = red_font

    else:
        day_cell.font = bold_font
        weekday_cell.font = bold_font

    #셀 스타일 적용
    for j in range(1,1 + max_index + 5):  ## 5:(일,요일,시간,순번빈칸) , 14일 매일 오전 오후 = 14*2 
        cell = ws.cell(row= j + (max_index + 5)*(i//28), column = 2 + i%28) 
        cell.alignment = center_aligned_text
    
    #순번셀 정렬
    if(i%28 == 0):
        for j in range(1,1 + max_index + 5):  ## 5:(일,요일,시간,순번빈칸), 14일 매일 오전 오후 = 14*2 
            cell = ws.cell(row= j + (max_index + 5)*(i//28), column = 1) 
            cell.alignment = center_aligned_text


# 열 너비 조정
for col in range(1, 8):
    ws.column_dimensions[get_column_letter(col)].width = 12

# 엑셀 파일로 저장
wb.save(output_path)

print("달력 형식의 배차 시간표22가 성공적으로 생성되었습니다.")




일  1                 2                   3        ...   29              30  \
요일  일       화         수         월         목       ...    목       일       금   
시간 오전 오후   오전   오후   오전   오후   오전   오후   오전   오후  ...   오전   오후 오전 오후   오전   
순번                                                ...                        
1         김현진  강형철  김현진  강형철  김현진  김민성  김현진  강형철  ...  강형철  김현진        강형철   
2         박준서  김민성  박준서  김종열  박준서  김종열  박준서  김민성  ...  김민성  박준서        김민성   
3         안병준  김진수  안병준  김진수  안병준  김진수  안병준  김종열  ...  김종열  안병준        김진수   
4         양귀선  민경오  양귀선  민경오  양귀선  민경오  양귀선  김진수  ...  김진수  양귀선        민경오   
5         이동규  이상준  이동규  이상준  이동규  이상준  이동규  이상준  ...  이상준  이동규        이상준   
6         이명재  조예찬  이명재  조예찬  이명재  조예찬  이명재  조예찬  ...  조예찬  이명재        조예찬   

일                 31     
요일         월       토     
시간   오후   오전   오후 오전 오후  
순번                       
1   김현진  김현진  강형철        
2   박준서  박준서  김민성        
3   안병준  안병준  김종열        
4   양귀선  양귀선  김진수        
5   이동규  이동

C:\Users\user\AppData\Local\Temp\ipykernel_12552\2202511477.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  schedule.fillna(value='',inplace=True)


In [2]:
## 근무시간정보 자동생성
import pandas as pd
import datetime

def generate_time_slots(start_date, days, shifts_per_period=6):
    """
    근무 시간 정보를 자동 생성하는 함수
    :param start_date: 시작 날짜 (datetime.date 형식)
    :param days: 생성할 총 일수
    :param shifts_per_period: 한 번의 시간대(오전/오후)에 근무할 순번 수
    :return: 근무 시간 정보 DataFrame
    """
    weekday_korean = {0:'월',1:'화',2:'수',3:'목',4:'금',5:'토',6:'일'}

    time_slots = []
    
    for i in range(days):
        current_date = start_date + datetime.timedelta(days=i)
        
        # 오전 시간대
        for j in range(1, shifts_per_period + 1):
            time_slots.append({'날짜': current_date, '시간': '오전', '순번': j,'월':current_date.month,'일':current_date.day,
                               '주차':current_date.isocalendar()[1],'요일':weekday_korean[current_date.weekday()]})
        
        # 오후 시간대
        for j in range(1, shifts_per_period + 1):
            time_slots.append({'날짜': current_date, '시간': '오후', '순번': j,'월':current_date.month,'일':current_date.day,
                               '주차':current_date.isocalendar()[1],'요일':weekday_korean[current_date.weekday()]})
    
    return pd.DataFrame(time_slots)

# 시작 날짜와 기간 설정
start_date = datetime.date(2024, 9, 1)
days = 31  # 예: 50일 동안의 근무 시간 생성

# 근무 시간 정보 생성
time_slots_df = generate_time_slots(start_date, days)

# 엑셀 파일로 저장
output_path = r'C:\Users\user\poi\PoiTransport\근무시간정보.xlsx'
time_slots_df.to_excel(output_path, index=False)

print("근무 시간 정보가 성공적으로 생성되었습니다.")

근무 시간 정보가 성공적으로 생성되었습니다.


In [1]:
## designer 실행방법: 터미널에 qt5-tools designer


SystemExit: 0

c:\Users\user\.conda\envs\POI_TRANSPORT\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
import os
import sys
from PyQt5.QtCore import QLibraryInfo

designer_path = os.path.join(QLibraryInfo.location(QLibraryInfo.BinariesPath), "designer.exe")
print("Qt Designer path:", designer_path)

Qt Designer path: C:/Users/user/.conda/envs/poi_transport/lib/site-packages/PyQt5/Qt5/bin\designer.exe
